# **I Know What You Will Do: Forecasting Motor Behaviour from EEG Time Series**

**[Brainhack Rome 2025](https://brainhackrome.github.io/) - Project #3** 

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/matteo-d-m/brainhack-rome-forecasting/blob/main/eeg-forecasting-notebook.ipynb)

In [2]:
from pathlib import Path

import numpy as np
import matplotlib.pyplot as plt

import mne

import torch
from torch.utils.data import Dataset, random_split